Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
import cPickle as pickle
import numpy as np
import tensorflow as tf

First reload the data we generated in _notmist.ipynb_.

In [3]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print 'Training set', train_dataset.shape, train_labels.shape
  print 'Validation set', valid_dataset.shape, valid_labels.shape
  print 'Test set', test_dataset.shape, test_labels.shape

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (18724, 28, 28) (18724,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [4]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print 'Training set', train_dataset.shape, train_labels.shape
print 'Validation set', valid_dataset.shape, valid_labels.shape
print 'Test set', test_dataset.shape, test_labels.shape

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (18724, 784) (18724, 10)


In [5]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compue the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

### Training the Logistic Model with L2 Regularization

Here the Beta value is set to 0.003, I observed the highest accuracy here and hence decided to go with it.

In [84]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():
  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))+0.003*tf.nn.l2_loss(weights)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [85]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print "Initialized"
  for step in xrange(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print "Minibatch loss at step", step, ":", l
      print "Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels)
      print "Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels)
  print "Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels)

Initialized
Minibatch loss at step 0 : 25.1584
Minibatch accuracy: 9.4%
Validation accuracy: 8.5%
Minibatch loss at step 500 : 2.44396
Minibatch accuracy: 75.8%
Validation accuracy: 77.8%
Minibatch loss at step 1000 : 1.1102
Minibatch accuracy: 78.9%
Validation accuracy: 81.2%
Minibatch loss at step 1500 : 0.794458
Minibatch accuracy: 79.7%
Validation accuracy: 81.6%
Minibatch loss at step 2000 : 0.696043
Minibatch accuracy: 85.2%
Validation accuracy: 81.9%
Minibatch loss at step 2500 : 0.796149
Minibatch accuracy: 85.2%
Validation accuracy: 81.3%
Minibatch loss at step 3000 : 0.800042
Minibatch accuracy: 79.7%
Validation accuracy: 81.0%
Test accuracy: 88.1%


### Training the Neural Network with L2 Regularization

In [88]:
batch_size = 128
hidden1_units = 1024
beta = 0.003

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Set hidden layer 1
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden1_units]))
  biases1 = tf.Variable(tf.zeros([hidden1_units]))
    
  hidden1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)

  # Set final layer variables
  weights = tf.Variable(
    tf.truncated_normal([hidden1_units, num_labels]))

  biases = tf.Variable(tf.zeros([num_labels]))
    
  logits = tf.matmul(hidden1, weights) + biases
    
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

  loss = loss + 0.5*beta*(tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)

  ## validation operation
  valid_prediction1 =  tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1)
  valid_prediction = tf.nn.softmax(
    tf.matmul(valid_prediction1, weights) + biases)
  

  #test operation
  
  test_prediction1 = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
  test_prediction = tf.nn.softmax(tf.matmul(test_prediction1, weights) + biases)

In [89]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print "Initialized"
  for step in xrange(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print "Minibatch loss at step", step, ":", l
      print "Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels)
      print "Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels)
  print "Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels)

Initialized
Minibatch loss at step 0 : 756.965
Minibatch accuracy: 14.1%
Validation accuracy: 22.3%
Minibatch loss at step 500 : 238.197
Minibatch accuracy: 77.3%
Validation accuracy: 79.4%
Minibatch loss at step 1000 : 103.797
Minibatch accuracy: 77.3%
Validation accuracy: 82.5%
Minibatch loss at step 1500 : 48.6497
Minibatch accuracy: 79.7%
Validation accuracy: 84.5%
Minibatch loss at step 2000 : 23.0662
Minibatch accuracy: 88.3%
Validation accuracy: 85.9%
Minibatch loss at step 2500 : 11.2348
Minibatch accuracy: 85.9%
Validation accuracy: 86.9%
Minibatch loss at step 3000 : 5.69191
Minibatch accuracy: 82.8%
Validation accuracy: 87.1%
Test accuracy: 93.0%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [92]:
batch_size = 5

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  loss = loss + 0.5*tf.nn.l2_loss(tf.nn.l2_normalize(weights, dim=0))
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [93]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print "Initialized"
  for step in xrange(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print "Minibatch loss at step", step, ":", l
      print "Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels)
      print "Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels)
  print "Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels)

Initialized
Minibatch loss at step 0 : 22.8141
Minibatch accuracy: 0.0%
Validation accuracy: 12.8%
Minibatch loss at step 500 : 3.81225
Minibatch accuracy: 80.0%
Validation accuracy: 71.1%
Minibatch loss at step 1000 : 2.53587
Minibatch accuracy: 100.0%
Validation accuracy: 71.1%
Minibatch loss at step 1500 : 6.14686
Minibatch accuracy: 40.0%
Validation accuracy: 73.2%
Minibatch loss at step 2000 : 6.1302
Minibatch accuracy: 80.0%
Validation accuracy: 70.5%
Minibatch loss at step 2500 : 3.60564
Minibatch accuracy: 80.0%
Validation accuracy: 70.5%
Minibatch loss at step 3000 : 2.50001
Minibatch accuracy: 100.0%
Validation accuracy: 76.5%
Test accuracy: 83.4%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [101]:
batch_size = 128
hidden1_units = 1024
beta = 0.003

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Set hidden layer 1
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden1_units]))
  biases1 = tf.Variable(tf.zeros([hidden1_units]))
    
  hidden1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
  hidden1=tf.nn.dropout(hidden1,0.6)

  # Set final layer variables
  weights = tf.Variable(
    tf.truncated_normal([hidden1_units, num_labels]))

  biases = tf.Variable(tf.zeros([num_labels]))
    
  logits = tf.matmul(hidden1, weights) + biases
    
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

  loss = loss + 0.5*beta*(tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)

  ## validation operation
  valid_prediction1 =  tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1)
  valid_prediction = tf.nn.softmax(
    tf.matmul(valid_prediction1, weights) + biases)
  
  test_prediction1 = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
  test_prediction = tf.nn.softmax(tf.matmul(test_prediction1, weights) + biases)

In [102]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print "Initialized"
  for step in xrange(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print "Minibatch loss at step", step, ":", l
      print "Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels)
      print "Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels)
  print "Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels)

Initialized
Minibatch loss at step 0 : 880.952
Minibatch accuracy: 9.4%
Validation accuracy: 34.5%
Minibatch loss at step 500 : 226.676
Minibatch accuracy: 76.6%
Validation accuracy: 78.0%
Minibatch loss at step 1000 : 104.955
Minibatch accuracy: 78.1%
Validation accuracy: 80.3%
Minibatch loss at step 1500 : 48.7082
Minibatch accuracy: 82.0%
Validation accuracy: 83.5%
Minibatch loss at step 2000 : 23.0307
Minibatch accuracy: 87.5%
Validation accuracy: 85.0%
Minibatch loss at step 2500 : 11.2736
Minibatch accuracy: 85.9%
Validation accuracy: 86.0%
Minibatch loss at step 3000 : 5.81657
Minibatch accuracy: 78.9%
Validation accuracy: 86.2%
Test accuracy: 92.3%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [111]:
hidden_size=1024
hidden2_size=300
batch_size=128
beta1=0.01
beta2=0.01
beta3=0.0001

graph=tf.Graph()
with graph.as_default():
    tf_train_dataset=tf.placeholder(np.float32,shape=(batch_size,image_size*image_size))
    tf_train_labels=tf.placeholder(np.float32,shape=(batch_size,num_labels))
    tf_valid_dataset=tf.constant(valid_dataset)
    tf_test_dataset=tf.constant(test_dataset)
    
    weights_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_size]))
    biases_1 = tf.Variable(tf.zeros([hidden_size]))
    
    hidden_1=tf.nn.relu(tf.matmul(tf_train_dataset,weights_1)+biases_1)
    
#     hidden_1=tf.nn.dropout(hidden_1,0.5)
    
    weights_2 = tf.Variable(
    tf.truncated_normal([hidden_size, hidden2_size]))
    biases_2 = tf.Variable(tf.zeros([hidden2_size]))
    
    hidden_2=tf.nn.relu(tf.matmul(hidden_1,weights_2)+biases_2)
    
#     hidden_2=tf.nn.dropout(hidden_2,0.5)
  
    weights_3 = tf.Variable(
    tf.truncated_normal([hidden2_size, num_labels]))
    biases_3 = tf.Variable(tf.zeros([num_labels]))
    
    logits=tf.matmul(hidden_2,weights_3)+biases_3
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))+beta1*tf.nn.l2_loss(weights_1)+beta2*tf.nn.l2_loss(weights_2)
    +beta3*tf.nn.l2_loss(weights_3)

    
    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.04, global_step,1000,0.90,staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
#     optimizer = tf.train.GradientDescentOptimizer(0.01).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1),weights_2)+biases_2),weights_3)
    +biases_3)
    test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1),weights_2)+biases_2),weights_3)
    +biases_3)

In [112]:
num_steps=9001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('initialized')
    for step in range(num_steps):
        offset=(step*batch_size)%(train_dataset.shape[0]-batch_size)
        
        batch_data=train_dataset[offset:(offset+batch_size),:]
        batch_labels=train_labels[offset:(offset+batch_size),:]
        
        feed_dict={tf_train_dataset:batch_data,tf_train_labels:batch_labels}
        
        _,l,predictions=session.run([optimizer,loss,train_prediction],feed_dict=feed_dict)
        
        if (step % 1000 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
            
        global_step+=1
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

initialized
Minibatch loss at step 0: 8160.714844
Minibatch accuracy: 5.5%
Validation accuracy: 22.0%
Minibatch loss at step 1000: 1941.215088
Minibatch accuracy: 44.5%
Validation accuracy: 50.0%
Minibatch loss at step 2000: 944.974121
Minibatch accuracy: 62.5%
Validation accuracy: 63.5%
Minibatch loss at step 3000: 494.764618
Minibatch accuracy: 71.9%
Validation accuracy: 70.5%
Minibatch loss at step 4000: 276.063965
Minibatch accuracy: 82.8%
Validation accuracy: 74.6%
Minibatch loss at step 5000: 163.584442
Minibatch accuracy: 79.7%
Validation accuracy: 77.6%
Minibatch loss at step 6000: 102.284477
Minibatch accuracy: 75.8%
Validation accuracy: 79.9%
Minibatch loss at step 7000: 66.865364
Minibatch accuracy: 86.7%
Validation accuracy: 81.5%
Minibatch loss at step 8000: 45.869137
Minibatch accuracy: 82.8%
Validation accuracy: 82.9%
Minibatch loss at step 9000: 32.538139
Minibatch accuracy: 84.4%
Validation accuracy: 83.8%
Test accuracy: 90.8%
